In [1]:
import json
import pandas as pd
import copy
import random
import numpy as np
import distutils.dir_util
import io
import os

In [3]:
def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname)
    distutils.dir_util.mkpath(parent)
    with io.open(fname, "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

def load_json(fname):
    with open(fname, encoding='utf-8') as f:
        json_obj = json.load(f)

    return json_obj

In [55]:
train_f = load_json('res/train_filter.json')
train = load_json('res/train.json')

In [56]:
train

[{'tags': ['락'],
  'id': 61281,
  'plylst_title': '여행같은 음악',
  'songs': [525514,
   129701,
   383374,
   562083,
   297861,
   139541,
   351214,
   650298,
   531057,
   205238,
   706183,
   127099,
   660493,
   461973,
   121455,
   72552,
   223955,
   324992,
   50104],
  'like_cnt': 71,
  'updt_date': '2013-12-19 18:36:19.000'},
 {'tags': ['추억', '회상'],
  'id': 10532,
  'plylst_title': '요즘 너 말야',
  'songs': [432406,
   675945,
   497066,
   120377,
   389529,
   244277,
   461062,
   696302,
   442765,
   532114,
   586541,
   33389,
   244000,
   692078,
   37741,
   645653,
   571802,
   200183,
   61435,
   204499,
   41749,
   129258,
   413920,
   117205,
   6546,
   152422,
   602724,
   425946,
   173634,
   631268,
   409869,
   97749,
   395416,
   103741,
   181101,
   472144,
   414721,
   75801,
   315216,
   192882,
   383960,
   548636],
  'like_cnt': 1,
  'updt_date': '2014-12-02 16:19:42.000'},
 {'tags': ['까페', '잔잔한'],
  'id': 76951,
  'plylst_title': '편하게, 잔잔하게 

In [57]:
train_f

{'tags': {'0': ['팝'],
  '1': ['기분좋은'],
  '2': ['겨울', '크리스마스'],
  '3': ['발라드'],
  '4': ['일렉'],
  '5': ['감성새벽', '보고싶다'],
  '6': ['발라드'],
  '7': ['한국대중음악상', '뮤지션'],
  '8': ['락'],
  '9': ['댄스'],
  '10': ['기분전환', 'Pop'],
  '11': ['일렉'],
  '12': ['2017', '신남', '모음집'],
  '13': ['월디페', 'electronica', 'EDM', '빌로우', '신나는'],
  '14': ['electronica', '빌로우', 'EDM', '신나는'],
  '15': ['연말', '추울때', '겨울', '따듯한', '12월', '쌀쌀한', '퇴근길', '찬바람'],
  '16': ['일렉'],
  '17': ['OST', 'etc', '한스짐머'],
  '18': ['힙합', '힙합엘이', '알앤비'],
  '19': ['발라드'],
  '20': ['락'],
  '21': ['electronica', '빌로우', 'EDM', '신나는'],
  '22': ['electronica', '빌로우', 'EDM', '신나는'],
  '23': ['힙합', '랩'],
  '24': ['힙합', '힙합엘이', '기분전환', 'Drake'],
  '25': ['일렉'],
  '26': ['60년대', '사이키델릭', 'Psychedelic', 'Rock'],
  '27': ['봄', '사랑'],
  '28': ['발라드'],
  '29': ['발라드'],
  '30': ['80년대', '추천', '가요', '모음'],
  '31': ['댄스'],
  '32': ['댄스'],
  '33': ['댄스'],
  '34': ['댄스'],
  '35': ['회상', '추억', 'OST', '그리움', '드라마', '90년대'],
  '36': ['댄스'],
  '37': ['댄스'],
  '38

- 해결 방안: orient = 'records'

---

In [4]:
random.seed(777)
playlists = load_json('res/train_filter.json')

In [5]:
random.shuffle(playlists)

In [6]:
# split data
tot = len(playlists)
tot

76309

In [7]:
train = playlists[:int(tot*0.80)]
val = playlists[int(tot*0.80):]

In [8]:
write_json(train, "arena_data/orig/train.json")
write_json(val, "arena_data/orig/val.json")

In [17]:
len(train), len(val)

(61047, 15262)

In [18]:
# mask_data
playlists = copy.deepcopy(val)
tot = len(playlists)
song_only = playlists[:int(tot * 0.3)]
song_and_tags = playlists[int(tot * 0.3):int(tot * 0.8)]
tags_only = playlists[int(tot * 0.8):int(tot * 0.95)]
title_only = playlists[int(tot * 0.95):]
print(f"Total: {len(playlists)}, "
              f"Song only: {len(song_only)}, "
              f"Song & Tags: {len(song_and_tags)}, "
              f"Tags only: {len(tags_only)}, "
              f"Title only: {len(title_only)}")

Total: 15262, Song only: 4578, Song & Tags: 7631, Tags only: 2289, Title only: 764


In [19]:
def mask(playlists, mask_cols, del_cols):
        q_pl = copy.deepcopy(playlists)
        a_pl = copy.deepcopy(playlists)

        for i in range(len(playlists)):
            for del_col in del_cols:
                q_pl[i][del_col] = []
                if del_col == 'songs':
                    a_pl[i][del_col] = a_pl[i][del_col][:100]
                elif del_col == 'tags':
                    a_pl[i][del_col] = a_pl[i][del_col][:10]

            for col in mask_cols:
                mask_len = len(playlists[i][col])
                mask = np.full(mask_len, False)
                mask[:mask_len//2] = True
                np.random.shuffle(mask)

                q_pl[i][col] = list(np.array(q_pl[i][col])[mask])
                a_pl[i][col] = list(np.array(a_pl[i][col])[np.invert(mask)])

        return q_pl, a_pl

In [20]:
song_q, song_a = mask(song_only, ['songs'], ['tags'])
songtag_q, songtag_a = mask(song_and_tags, ['songs', 'tags'], [])
tag_q, tag_a = mask(tags_only, ['tags'], ['songs'])
title_q, title_a = mask(title_only, [], ['songs', 'tags'])

In [21]:
q = song_q + songtag_q + tag_q + title_q
a = song_a + songtag_a + tag_a + title_a

shuffle_indices = np.arange(len(q))
np.random.shuffle(shuffle_indices)

val_q = list(np.array(q)[shuffle_indices])
val_a = list(np.array(a)[shuffle_indices])

In [22]:
write_json(val_q, "arena_data/questions/val.json")
write_json(val_a, "arena_data/answers/val.json")

In [23]:
len(val_q), len(val_a)

(15262, 15262)